# ERA5 Heatwave

In [1]:
import numpy as np
%pylab inline
import netCDF4
from netCDF4 import Dataset
from datetime import datetime
from scipy import signal
#from pylab import *
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import *
from matplotlib.cm import ScalarMappable
import time

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


# 1.  Download data from ERA5 

Use ERA5_month_download.py to download monthly separated data. 
Set area, path for download and time inside the script.

In [ ]:
! python3 ERA5_month_download.py

In [2]:
confind = 95 # 95% above 3 consecutives days

# 2. ERA5 combine expver1 +expver5¶

In [ ]:
ERA5 = xr.open_mfdataset(path+'ERA5_monthly_12_8123.nc',combine='by_coords')
ERA5_combine =ERA5.sel(expver=1).combine_first(ERA5.sel(expver=5))
ERA5_combine.load()
ERA5_combine.to_netcdf(path+"ERA5_monthly_12_8123_expver_combine.nc")


# 3. Find Heatwave inside months blocks

In [5]:
def month_block(path,var,month):
    """ERA5 hourly data to months blocks
    """
    f = Dataset(path)
    nc = f.variables
    if month == '31':  
        bsize = 31*24   #(31 days = 744 hours) 
        t,la,lo=nc[var].shape
        print(nc[var].shape)
        lmonth= int(t/bsize)
    elif month == '30':
        bsize = 30*24   #(30 days = 720 hours) 
        t,la,lo=nc[var].shape
        print(nc[var].shape)
        lmonth= int(t/bsize) 
    elif month == '28':
        bsize = 28*24   #(30 days = 720 hours) 
        t,la,lo=nc[var].shape
        print(nc[var].shape)
        lmonth= int(t/bsize)
    elif month == '29':
        bsize = 30*24   #(30 days = 720 hours) 
        t,la,lo=nc[var].shape
        print(nc[var].shape)
        lmonth= int(t/bsize)
    x, u,v={},{},{}
    x_mean = ma.masked_all((lmonth,la,lo))
    for i in range (lmonth):
        if var == 't2m':
            x[i] = nc[var][i*bsize:(i+1)*bsize]- 273.15 # from K to C
            x_mean[i] = x[i].mean(axis=0)     
        elif var == 'tp':
            x[i] = nc[var][i*bsize:(i+1)*bsize]*1000*24 # from m to mm
            x_mean[i] = x[i].mean(axis=0) 
        elif var == 'd2m':
            x[i] = nc[var][i*bsize:(i+1)*bsize]- 273.15 # from K to C
            x_mean[i] = x[i].mean(axis=0) 
        elif var == 'u10':
            u[i] = nc['u10'][i*bsize:(i+1)*bsize]*3.6  #m/s to km/h
            v[i] = nc['v10'][i*bsize:(i+1)*bsize]*3.6  #m/s to km/h
            x[i] = np.sqrt((u[i]**2) + (v[i]**2))
            x_mean[i] = x[i].mean(axis=0) 
            

    return x,x_mean

def month_block_feb(path,var):
    """ERA5 hourly data to months blocks
    """
    f = Dataset(path)
    nc = f.variables
    t,la,lo=nc[var].shape
    biss = [28,28,28,29]*10
    biss = biss+[28] #add 2021
    x ={}
    
    for i in biss:
        
        bsize = i *24
        lmonth= int(t/bsize)
        x_mean = ma.masked_all((lmonth,la,lo))
        for i in range(lmonth):
             if var == 't2m':
                x[i] = nc[var][i*bsize:(i+1)*bsize]- 273.15 # from K to C
                x_mean[i] = x[i].mean(axis=0) 
    return x,x_mean

def daily_block (var,stats):
    """Daily block average
    Input
    ------
    var : array
        hourly data
    stats : string
        choose between mean and max
    """
    #print(stats)
    t,la,lo=var.shape
    bsize = 24      # 24 hours = 1 day
    lmonth= int(t/24)
    x = ma.masked_all((lmonth,la,lo))
    
    for i in range (lmonth):
        if stats == 'max':
            x[i]  = var[i*bsize:(i+1)*bsize].max(axis=0) #1day window find max temp
            #x[i].mean(axis=1).mean(axis=1)  
        elif stats == 'mean':
            x[i] = var[i*bsize:(i+1)*bsize].mean(axis=0)
        elif stats =='min':
            x[i]  = var[i*bsize:(i+1)*bsize].min(axis=0)            
            
        #from sklearn.linear_model import LinearRegression
        #model = LinearRegression()
        #model.fit(time,set20)
        #trend=model.predict(set20)
        else:
            print("Choose between 'mean', 'max' and 'sum'")
                                                     
    return x #signal.detrend(x)

def climatology(temp):
    days,lat,lon = temp[0].shape
    years = len(temp.keys())
    #years =41 #30 1981-2010
    ts=ma.masked_all((years,days))
    sp = ma.masked_all((years,lat,lon))
    clim = np.zeros(days)
    for i in range(years):
        ts[i] = temp[i].mean(axis=1).mean(axis=1)
    # range clim for each day
    for i in range(years):
        for j in range(days):
            clim[j] = clim[j]+ts[i][j]
            #print (i,j,ts[i][j], clim[j])

    return clim/years


def HeatWave_ts (ts,clim95):
    """Heat wave 
       periods of consecutive days with Tmax values above a certain percentile of 
       Tmax for the particular calendar day (calculated on a 15 day window). 
       Different percentiles (80th, 90th, 95th) and durations (3–4 days, 5–7 days, >7 days) 
       return:
       np.mean(HW) = mean temp of a HW
       np.sum(htwv_amp) = total days under HW
       len(htwv_amp) = frequency of HW
       np.mean(htwv_amp) = duration of HW
    """
    # found all heat waves above 95 percentil
    hw_where = np.where(ts>clim95)
    heat_wave =np.zeros(len(ts))
    for i in range (len(ts)):
        if ts[i]>clim95[i]:
            heat_wave[i]= ts[i]
    #search for consecutives days
    prev=np.zeros(len(hw_where[0])+1)             # plus 1 to add last group of heatwave
    count=0 
    low=np.zeros(len(hw_where[0]))
    for i in range (1,len(hw_where[0])):
        if hw_where[0][i] == hw_where[0][i-1]+1:  #loop each index above confind
            count+=1                              #if days consecutive add 1
            prev[i]=count 
        else:
            count =0                              #if day is not consecutive =0
    #search for zeros to found the separate classes of heatwaves
    low = []
    low = np.where(prev==0)[0] #SEARCH FOR EMPTY SPACES
    htwv_amp,HW=[],[]

    for i in range (1,len(low)):
        above95 = hw_where[0][low[i-1]:low[i]] 
        if i == len(low):
            above95 =(hw_where[0][low[i-1]:len(low)])
        if (len(above95)>=3):
            htwv_amp.append(len(above95)) # HW amplitude
            HW.append(np.max(heat_wave[above95]))
            
        else:   
            htwv_amp.append(0)
            HW.append(0)
            #htwv_count=len(htwv_amp)
        #elif (len(above95)<=7)&(len(above95)>=5): #can separate in categories 5-7 days
        #    htwv2.append(above95)
        #elif (len(above95)>=7): # category >7 days
        #    htwv3.append(above95)  
    return np.mean(HW),np.sum(htwv_amp), len(htwv_amp), np.mean(htwv_amp) 

def HW_dict(index):
    path= '/Users/calim/code/CEMADEN/'
    f = Dataset(path+'ERA5_Sept_2020.nc') #JUST TO PICK LAT AND LON
    nc = f.variables
    time,lat,lon = nc['t2m'].shape 
    HWjan,HWfeb,HWmar,HWapr =ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon))
    HWmay,HWjun,HWjul,HWaug = ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon))
    HWsep,HWoct,HWnov,HWdec = ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon))
 
    
    hwjan,hwfeb,hwmar,hwapr =ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon))
    hwmay,hwjun,hwjul,hwaug = ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon))
    hwsep,hwoct,hwnov,hwdec = ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon))
 
    ahwjan,ahwfeb,ahwmar,ahwapr =ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon))
    ahwmay,ahwjun,ahwjul,ahwaug = ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon))
    ahwsep,ahwoct,ahwnov,ahwdec = ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon))
 
    shwjan,shwfeb,shwmar,shwapr =ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon))
    shwmay,shwjun,shwjul,shwaug = ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon))
    shwsep,shwoct,shwnov,shwdec = ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon)),ma.masked_all((lat,lon))
 


    #index=38

    for i in range(lat):
        for j in range (lon):
                HWjan[i,j],shwjan[i,j],hwjan[i,j],ahwjan[i,j] = HeatWave_ts(tmax1[index][:,i,j],clim_95_1)
                HWfeb[i,j],shwfeb[i,j],hwfeb[i,j],ahwfeb[i,j] = HeatWave_ts(tmax2[index][:,i,j],clim_95_2)
                HWmar[i,j],shwmar[i,j],hwmar[i,j],ahwmar[i,j] = HeatWave_ts(tmax3[index][:,i,j],clim_95_3)
                HWapr[i,j],shwapr[i,j],hwapr[i,j],ahwapr[i,j] = HeatWave_ts(tmax4[index][:,i,j],clim_95_4)
                HWmay[i,j],shwmay[i,j],hwmay[i,j],ahwmay[i,j] = HeatWave_ts(tmax5[index][:,i,j],clim_95_5)
                HWjun[i,j],shwjun[i,j],hwjun[i,j],ahwjun[i,j] = HeatWave_ts(tmax6[index][:,i,j],clim_95_6)
                HWjul[i,j],shwjul[i,j],hwjul[i,j],ahwjul[i,j] = HeatWave_ts(tmax7[index][:,i,j],clim_95_7)
                HWaug[i,j],shwaug[i,j],hwaug[i,j],ahwaug[i,j] = HeatWave_ts(tmax8[index][:,i,j],clim_95_8)            
                HWsep[i,j],shwsep[i,j],hwsep[i,j],ahwsep[i,j] = HeatWave_ts(tmax9[index][:,i,j],clim_95_9)
                HWoct[i,j],shwoct[i,j],hwoct[i,j],ahwoct[i,j] = HeatWave_ts(tmax10[index][:,i,j],clim_95_10)
                HWnov[i,j],shwnov[i,j],hwnov[i,j],ahwnov[i,j] = HeatWave_ts(tmax11[index][:,i,j],clim_95_11)
                HWdec[i,j],shwdec[i,j],hwdec[i,j],ahwdec[i,j] = HeatWave_ts(tmax12[index][:,i,j],clim_95_12)              

                #sum_hw19[i,j] = HeatWave_ts(tmax10[38][:,i,j],clim_95_10)
                #sum_hw20[i,j] = HeatWave_ts(tmax10[39][:,i,j],clim_95_10)
                #sum_hw21[i,j] = HeatWave_ts(tmax10[40][:,i,j],clim_95_10)
    HW ={}
    HW[0]= [] 
    HW[1] =HWjan
    HW[2]= HWfeb
    HW[3]= HWmar
    HW[4]= HWapr
    HW[5]= HWmay
    HW[6]= HWjun
    HW[7]= HWjul
    HW[8]= HWaug
    HW[9]= HWsep
    HW[10]= HWoct
    HW[11]= HWnov
    HW[12]= HWdec
                
    hw ={}
    hw[0]= [] 
    hw[1] =hwjan
    hw[2]= hwfeb
    hw[3]= hwmar
    hw[4]= hwapr
    hw[5]= hwmay
    hw[6]= hwjun
    hw[7]= hwjul
    hw[8]= hwaug
    hw[9]= hwsep
    hw[10]= hwoct
    hw[11]= hwnov
    hw[12]= hwdec

    ahw ={}
    ahw[0]= [] 
    ahw[1] =ahwjan
    ahw[2]= ahwfeb
    ahw[3]= ahwmar
    ahw[4]= ahwapr
    ahw[5]= ahwmay
    ahw[6]= ahwjun
    ahw[7]= ahwjul
    ahw[8]= ahwaug
    ahw[9]= ahwsep
    ahw[10]= ahwoct
    ahw[11]= ahwnov
    ahw[12]= ahwdec
    
    
    shw ={}
    shw[0]= [] 
    shw[1] =shwjan
    shw[2]= shwfeb
    shw[3]= shwmar
    shw[4]= shwapr
    shw[5]= shwmay
    shw[6]= shwjun
    shw[7]= shwjul
    shw[8]= shwaug
    shw[9]= shwsep
    shw[10]= shwoct
    shw[11]= shwnov
    shw[12]= shwdec
    return HW,hw,ahw,shw


In [ ]:

f = Dataset(path+'ERA5_01_8123.nc') #JUST TO PICK LAT AND LON
nc = f.variables
time,lat,lon = nc['t2m'].shape 

sum_hw19 = ma.masked_all((lat,lon))
sum_hw20 = ma.masked_all((lat,lon))
sum_hw21 = ma.masked_all((lat,lon))

HW2018,hw2018,ahw2018,shw2018 = HW_dict(37)
HW2019,hw2019,ahw2019,shw2019 = HW_dict(38)
HW2020,hw2020,ahw2020,shw2020 = HW_dict(39)
HW2021,hw2021,ahw2021,shw2021 = HW_dict(40)
HW2022,hw2022,ahw2022,shw2022 = HW_dict(41)
#hw2022 = HW_dict(37)

HW1921={}
HW1921[0]=HW2018[12]
HW1921[1] =HW2019[1]
HW1921[2]= HW2019[2]
HW1921[3]= HW2019[3]
HW1921[4]= HW2019[4]
HW1921[5]= HW2019[5]
HW1921[6]= HW2019[6]
HW1921[7]= HW2019[7]
HW1921[8]= HW2019[8]
HW1921[9]= HW2019[9]
HW1921[10]= HW2019[10]
HW1921[11]= HW2019[11]
HW1921[12]= HW2019[12]

HW1921[13]= HW2020[1]
HW1921[14]= HW2020[2]
HW1921[15]= HW2020[3]
HW1921[16]= HW2020[4]
HW1921[17]= HW2020[5]
HW1921[18]= HW2020[6]
HW1921[19]= HW2020[7]
HW1921[20]= HW2020[8]
HW1921[21]= HW2020[9]
HW1921[22]= HW2020[10]
HW1921[23]= HW2020[11]
HW1921[24]= HW2020[12]

HW1921[25]= HW2021[1]
HW1921[26]= HW2021[2]
HW1921[27]= HW2021[3]
HW1921[28]= HW2021[4]
HW1921[29]= HW2021[5]
HW1921[30]= HW2021[6]
HW1921[31]= HW2021[7]
HW1921[32]= HW2021[8]
HW1921[33]= HW2021[9]
HW1921[34]= HW2021[10]
HW1921[35]= HW2021[11]
HW1921[36]= HW2021[12]
HW1921[37]= HW2022[1]
HW1921[38]= HW2022[2]






hw1921={}
hw1921[0]=hw2018[12]
hw1921[1] =hw2019[1]
hw1921[2]= hw2019[2]
hw1921[3]= hw2019[3]
hw1921[4]= hw2019[4]
hw1921[5]= hw2019[5]
hw1921[6]= hw2019[6]
hw1921[7]= hw2019[7]
hw1921[8]= hw2019[8]
hw1921[9]= hw2019[9]
hw1921[10]= hw2019[10]
hw1921[11]= hw2019[11]
hw1921[12]= hw2019[12]

hw1921[13]= hw2020[1]
hw1921[14]= hw2020[2]
hw1921[15]= hw2020[3]
hw1921[16]= hw2020[4]
hw1921[17]= hw2020[5]
hw1921[18]= hw2020[6]
hw1921[19]= hw2020[7]
hw1921[20]= hw2020[8]
hw1921[21]= hw2020[9]
hw1921[22]= hw2020[10]
hw1921[23]= hw2020[11]
hw1921[24]= hw2020[12]

hw1921[25]= hw2021[1]
hw1921[26]= hw2021[2]
hw1921[27]= hw2021[3]
hw1921[28]= hw2021[4]
hw1921[29]= hw2021[5]
hw1921[30]= hw2021[6]
hw1921[31]= hw2021[7]
hw1921[32]= hw2021[8]
hw1921[33]= hw2021[9]
hw1921[34]= hw2021[10]
hw1921[35]= hw2021[11]
hw1921[36]= hw2021[12]
hw1921[37]= hw2022[1]
hw1921[38]= hw2022[2]



ahw1921={}
ahw1921[0]= ahw2018[12]
ahw1921[1] = ahw2019[1]
ahw1921[2]= ahw2019[2]
ahw1921[3]= ahw2019[3]
ahw1921[4]= ahw2019[4]
ahw1921[5]= ahw2019[5]
ahw1921[6]= ahw2019[6]
ahw1921[7]= ahw2019[7]
ahw1921[8]= ahw2019[8]
ahw1921[9]= ahw2019[9]
ahw1921[10]= ahw2019[10]
ahw1921[11]= ahw2019[11]
ahw1921[12]= ahw2019[12]

ahw1921[13]= ahw2020[1]
ahw1921[14]= ahw2020[2]
ahw1921[15]= ahw2020[3]
ahw1921[16]= ahw2020[4]
ahw1921[17]= ahw2020[5]
ahw1921[18]= ahw2020[6]
ahw1921[19]= ahw2020[7]
ahw1921[20]= ahw2020[8]
ahw1921[21]= ahw2020[9]
ahw1921[22]= ahw2020[10]
ahw1921[23]= ahw2020[11]
ahw1921[24]= ahw2020[12]

ahw1921[25]= ahw2021[1]
ahw1921[26]= ahw2021[2]
ahw1921[27]= ahw2021[3]
ahw1921[28]= ahw2021[4]
ahw1921[29]= ahw2021[5]
ahw1921[30]= ahw2021[6]
ahw1921[31]= ahw2021[7]
ahw1921[32]= ahw2021[8]
ahw1921[33]= ahw2021[9]
ahw1921[34]= ahw2021[10]
ahw1921[35]= ahw2021[11]
ahw1921[36]= ahw2021[12]
ahw1921[37]= ahw2022[1]
ahw1921[38]= ahw2022[2]



shw1921={}
shw1921[0]= shw2018[12]/31
shw1921[1] = shw2019[1]/31
shw1921[2]= shw2019[2]/28
shw1921[3]= shw2019[3]/31
shw1921[4]= shw2019[4]/30
shw1921[5]= shw2019[5]/31
shw1921[6]= shw2019[6]/30
shw1921[7]= shw2019[7]/31
shw1921[8]= shw2019[8]/31
shw1921[9]= shw2019[9]/30
shw1921[10]= shw2019[10]/31
shw1921[11]= shw2019[11]/30
shw1921[12]= shw2019[12]/31

shw1921[13]= shw2020[1]/31
shw1921[14]= shw2020[2]/29
shw1921[15]= shw2020[3]/31
shw1921[16]= shw2020[4]/30
shw1921[17]= shw2020[5]/31
shw1921[18]= shw2020[6]/30
shw1921[19]= shw2020[7]/31
shw1921[20]= shw2020[8]/31
shw1921[21]= shw2020[9]/30
shw1921[22]= shw2020[10]/31
shw1921[23]= shw2020[11]/30
shw1921[24]= shw2020[12]/31

shw1921[25]= shw2021[1]/31
shw1921[26]= shw2021[2]/28
shw1921[27]= shw2021[3]/31
shw1921[28]= shw2021[4]/30
shw1921[29]= shw2021[5]/31
shw1921[30]= shw2021[6]/30
shw1921[31]= shw2021[7]/31
shw1921[32]= shw2021[8]/31
shw1921[33]= shw2021[9]/30
shw1921[34]= shw2021[10]/31
shw1921[35]= shw2021[11]/30
shw1921[36]= shw2021[12]/31
shw1921[37]= shw2022[1]/31
shw1921[38]= shw2022[2]/28



import pickle

pickle.dump(hw1921, open( path+"hw1921.p", "wb" ) )
pickle.dump(ahw1921, open( path+"ahw1921.p", "wb" ) )
pickle.dump(shw1921, open( path+"shw1921.p", "wb" ) )
pickle.dump(HW1921, open( path+"HW1921.p", "wb" ) )
pickle.dump(clim95, open( path+"clim95.p", "wb" ) )
